**Bank Nifty Prediction -LSTM**



In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator


In [ ]:
filename = "/content/Bankniftydata.csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          1366 non-null   object 
 1   Open          1366 non-null   float64
 2   High          1366 non-null   float64
 3   Low           1366 non-null   float64
 4   Close         1366 non-null   float64
 5   (High-Low)    1366 non-null   object 
 6   (Open-Close)  1366 non-null   object 
dtypes: float64(4), object(3)
memory usage: 74.8+ KB
None


In [ ]:
df.describe()

,Open,High,Low,Close
count,1366.000000,1366.000000,1366.000000,1366.000000
mean,25069.020644,25257.013433,24838.185359,25049.523609
std,5297.711750,5325.006303,5259.421152,5292.826117
min,13715.100000,13844.450000,13407.250000,13555.700000
25%,20771.112500,20956.262500,20520.725000,20678.187500
50%,25240.650000,25411.725000,25039.675000,25209.750000
75%,29054.100000,29356.700000,28844.612500,29036.425000
max,37492.600000,37708.750000,36764.250000,37306.250000


In [ ]:
df.head()

,Date,Open,High,Low,Close,(High-Low),(Open-Close)
0,2016-01-01,16932.50,17067.45,16824.05,17039.25,243.4,-106.75
1,2016-01-04,16966.35,16966.35,16575.45,16599.15,390.9,367.2
2,2016-01-05,16652.05,16670.20,16474.85,16542.50,195.35,109.55
3,2016-01-06,16505.45,16614.95,16377.65,16433.15,237.3,72.3
4,2016-01-07,16256.55,16256.55,16049.75,16073.85,206.8,182.7


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low','(High-Low)','(Open-Close)'], inplace=True)

In [ ]:
import plotly.graph_objects as go
trace = go.Scatter(
    x = df['Date'],
    y = df['Close'],
    mode = 'lines',
    name = 'Data'
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()

#Data Preprocessing
For our analysis, let train the model on the first 80% of data and test it on the remaining 20%.

In [ ]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.8
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

1092
274


#Luckily, there is a module in Keras that does exactly this: TimeseriesGenerator. Please look up the documentation for more info

I’ve set look_back as 3, but you can play around with that value.

In [ ]:
look_back = 3

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

#A simple architecture of LSTM units trained using Adam optimizer and Mean Squared Loss function for 25 epochs. Note that instead of using model.fit(), we use model.fit_generator() because we have created a data generator.

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.models import Model

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
#Model.fit(train_generator, epochs=num_epochs, verbose=1)
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



Epoch 1/25
55/55 [==============================] - 2s 2ms/step - loss: 607764521.7143
Epoch 2/25
55/55 [==============================] - 0s 2ms/step - loss: 380380380.5714
Epoch 3/25
55/55 [==============================] - 0s 3ms/step - loss: 201449052.2857
Epoch 4/25
55/55 [==============================] - 0s 3ms/step - loss: 67785977.0714
Epoch 5/25
55/55 [==============================] - 0s 3ms/step - loss: 10069995.4107
Epoch 6/25
55/55 [==============================] - 0s 3ms/step - loss: 1078352.7891
Epoch 7/25
55/55 [==============================] - 0s 3ms/step - loss: 368394.2854
Epoch 8/25
55/55 [==============================] - 0s 3ms/step - loss: 233109.8306
Epoch 9/25
55/55 [==============================] - 0s 3ms/step - loss: 189587.0060
Epoch 10/25
55/55 [==============================] - 0s 3ms/step - loss: 220059.3747
Epoch 11/25
55/55 [==============================] - 0s 3ms/step - loss: 222736.6109
Epoch 12/25
55/55 [==============================] - 0s 3ms/

#Prediction
Now that we have completed training, let us see if the network performed well. We can test the model on testing data and see if the prediction and the actual values overlap.

In [ ]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Bank Nifty Index",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1976: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [ ]:
mae = np.mean(np.abs(prediction))
print(mae)

28861.852


In [ ]:
mae = np.mean(np.abs(close_test))
print(mae)

28896.618248175182


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
expected = [28896.618248175182]
predictions = [28861.852]
mse = mean_squared_error(expected, predictions)

rmse = sqrt(mse)
print('RMSE:%f' %rmse)

RMSE:34.766248


In [ ]:
ActualData FittedValue AbsolutePercentageError
120        119.5       (abs(120-119)/120)*100 = 0.83%

In [ ]:
Accuracy=(34.766248/28896.61825)*100

In [ ]:
Accuracy

0.120312514423725

In [ ]:
prediction

array([20799.918, 20825.363, 20437.584, 20345.572, 20091.766, 20401.574,
       20672.434, 21230.164, 21644.41 , 21823.36 , 21842.46 , 21484.127,
       21500.932, 21471.035, 21488.621, 21679.992, 21942.158, 21970.43 ,
       22119.922, 22411.754, 22670.125, 22645.646, 22576.033, 22091.188,
       21713.326, 21416.596, 21549.06 , 21861.447, 22245.166, 22580.615,
       22874.078, 22898.31 , 22693.78 , 22289.107, 21974.672, 21999.533,
       21849.26 , 21524.04 , 21428.643, 21292.494, 21526.258, 21600.324,
       21729.082, 21896.   , 22076.438, 22230.482, 22119.924, 21929.377,
       21785.875, 21967.744, 22169.574, 22196.705, 22264.336, 22629.338,
       23032.695, 23296.498, 23700.107, 23920.564, 24132.348, 23795.21 ,
       23771.406, 23585.84 , 23244.26 , 22933.348, 22739.719, 22534.72 ,
       22371.773, 22393.117, 22355.922, 22311.637, 22467.6  , 22379.768,
       22030.256, 21635.791, 21254.45 , 21008.664, 20909.176, 20870.398,
       21284.197, 21539.52 , 21594.832, 21889.871, 

From the graph, we can see that prediction and the actual value(ground truth) somewhat overlap. But if you zoom in, the fit is not perfect. We should expect this because it is inevitable as we are performing prediction.

#Forecasting
Our testing shows the model is somewhat good. So we can move on to predicting the future or forecasting.
Foreshadowing: Since we are attempting to predict the future, there will be a great amount of uncertainty in the prediction.
Predicting the future is easy… To predict tomorrow's value, feed into the model the past n(look_back) days’ values and we get tomorrow’s value as output. To get the day after tomorrow’s value, feed-in past n-1 days’ values along with tomorrow’s value and the model output day after tomorrow’s value.
Forecasting for longer duration is not feasible. So, let’s forecast a months stock price.

In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [ ]:
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'train'
)
trace2 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'test'
)
trace3 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode='lines',
    name = 'Forecast'

)
layout = go.Layout(
    title = "Bank Nifty Index",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [ ]:
forecast

array([35071.95      , 35466.48828125, 35246.55078125, 35228.44140625,
       35344.31640625, 35257.33203125, 35267.1328125 , 35297.24609375,
       35264.93359375, 35272.41015625, 35278.203125  , 35266.29296875,
       35269.2265625 , 35268.86328125, 35264.0078125 , 35264.29296875,
       35262.65234375, 35260.08203125, 35259.1640625 , 35257.3828125 ,
       35255.49609375, 35254.109375  , 35252.375     , 35250.6796875 ,
       35249.12109375, 35247.4296875 , 35245.78125   , 35244.16015625,
       35242.5       , 35240.85546875, 35239.21875   ])

#Predicting the future is easy… Is it?
When predicting the future, there is a good possibility that model output is uncertain to a great extent. The model’s output is fed back into it as input. This causes the model’s noise and uncertainty to be repeated and amplified.
But still, we have created a model that gives us a trend of the graphs and also the range of values that might be in the future.

#Conclusion
With this model, we have created a rudimentary model that is able to forecast to a certain extent. Even though the model is not perfect, we have one that can approximate to the past data pretty well. But for new data, we would require more parameter tuning.
If there exists any problem during the training of the model, try tuning these parameters:
look_back
batch size
LSTM units
num_epochs